In [2]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import kstest, normaltest, jarque_bera, kurtosis, skew
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# STEP 1: LOAD DATA
# ============================================================================

def load_stock_data(symbols, timeframe=5):
    """Load data for multiple stocks"""
    stock_data = {}
    
    for symbol in symbols:
        try:
            df = pd.read_parquet(
                f"s3://quant-ohlcv-data/ohlcv/timeframe={timeframe}min/symbol={symbol}/"
            )
            if "datetime" in df.columns:
                df = df.set_index("datetime")
            
            start_date = pd.Timestamp("2023-02-01", tz="Asia/Kolkata")
            end_date = pd.Timestamp("2025-12-31", tz="Asia/Kolkata")
            df = df.loc[start_date:end_date]
            
            stock_data[symbol] = df
            print(f"✓ Loaded {symbol}: {len(df)} candles")
        except Exception as e:
            print(f"✗ Failed to load {symbol}: {e}")
    
    return stock_data

# symbols = ["ADANIPORTS", "AXISBANK", "HINDALCO", "ICICIBANK", 
#            "INFY", "M&M", "RELIANCE", "POWERGRID", "SBIN", "HDFCBANK"]
symbols = ["ADANIPORTS", "AXISBANK", "HINDALCO", "ICICIBANK", 
           "INFY", "M&M", "RELIANCE", "POWERGRID", "SBIN", "HDFCBANK"]

print("="*80)
print("LOADING STOCK DATA")
print("="*80)
stock_data = load_stock_data(symbols)

# ============================================================================
# STEP 2: CALCULATE FEATURES
# ============================================================================

def calculate_rsi(prices, period=14):
    """Calculate RSI"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_bollinger_position(prices, window=20):
    """Calculate position within Bollinger Bands (0-1 scale)"""
    ma = prices.rolling(window).mean()
    std = prices.rolling(window).std()
    upper = ma + 2 * std
    lower = ma - 2 * std
    position = (prices - lower) / (upper - lower)
    return position.clip(0, 1)

def calculate_features(df, symbol):
    """Calculate all relevant features for a stock"""
    df = df.copy()
    df['symbol'] = symbol
    df['year'] = df.index.year
    
    # Price-based features
    df['returns'] = df['close'].pct_change()
    df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
    df['range'] = df['high'] - df['low']
    df['range_pct'] = (df['range'] / df['close']) * 100
    
    # Volatility features
    df['h-l'] = df['high'] - df['low']
    df['h-pc'] = abs(df['high'] - df['close'].shift(1))
    df['l-pc'] = abs(df['low'] - df['close'].shift(1))
    df['tr'] = df[['h-l', 'h-pc', 'l-pc']].max(axis=1)
    df['atr_14'] = df['tr'].rolling(window=14).mean()
    df['atr_50'] = df['tr'].rolling(window=50).mean()
    
    # Support/Resistance levels
    df['support_10'] = df['close'].rolling(window=10).min()
    df['resistance_10'] = df['close'].rolling(window=10).max()
    df['sr_range'] = df['resistance_10'] - df['support_10']
    df['sr_range_pct'] = (df['sr_range'] / df['close']) * 100
    df['distance_from_support'] = (df['close'] - df['support_10']) / df['close'] * 100
    df['distance_from_resistance'] = (df['resistance_10'] - df['close']) / df['close'] * 100
    
    # Volume features
    df['volume_ma_20'] = df['volume'].rolling(window=20).mean()
    df['volume_ratio'] = df['volume'] / df['volume_ma_20']
    
    # Momentum features
    df['rsi_14'] = calculate_rsi(df['close'], 14)
    df['momentum_5'] = df['close'].pct_change(5)
    df['momentum_20'] = df['close'].pct_change(20)
    
    # Volatility regime
    df['volatility_percentile'] = df['atr_14'].rolling(window=252).apply(
        lambda x: stats.percentileofscore(x, x.iloc[-1]) if len(x) > 0 else np.nan
    )
    
    # Mean reversion indicators
    df['distance_from_ma_20'] = (df['close'] - df['close'].rolling(20).mean()) / df['close'] * 100
    df['bollinger_position'] = calculate_bollinger_position(df['close'])
    
    return df

print("\n" + "="*80)
print("CALCULATING FEATURES")
print("="*80)

enriched_data = {}
for symbol, df in stock_data.items():
    enriched_data[symbol] = calculate_features(df, symbol)
    print(f"✓ Calculated features for {symbol}")

# ============================================================================
# STEP 3: STOCK CHARACTERIZATION
# ============================================================================

def calculate_hurst_exponent(prices, max_lag=20):
    """Calculate Hurst Exponent"""
    if len(prices) < 100:
        return np.nan
    
    lags = range(2, min(max_lag, len(prices) // 2))
    tau = []
    
    for lag in lags:
        pp = np.subtract(prices[lag:], prices[:-lag])
        tau.append(np.std(pp))
    
    try:
        if len(tau) > 0:
            poly = np.polyfit(np.log(lags), np.log(tau), 1)
            return poly[0]
        else:
            return np.nan
    except:
        return np.nan

def calculate_half_life(returns):
    """Calculate half-life of mean reversion"""
    if len(returns) < 50:
        return np.nan
    
    returns_lag = np.roll(returns, 1)
    returns_lag[0] = 0
    returns_diff = returns - returns_lag
    
    try:
        from scipy import stats as sp_stats
        slope, _, _, _, _ = sp_stats.linregress(returns_lag[1:], returns_diff[1:])
        
        if slope >= 0:
            return np.nan
        
        half_life = -np.log(2) / slope
        return half_life if half_life > 0 and half_life < 1000 else np.nan
    except:
        return np.nan

def calculate_price_efficiency(prices):
    """Calculate price movement efficiency"""
    if len(prices) < 10:
        return np.nan
    
    direct_distance = abs(prices[-1] - prices[0])
    actual_distance = np.sum(np.abs(np.diff(prices)))
    
    if actual_distance == 0:
        return np.nan
    
    return direct_distance / actual_distance

def calculate_regime_persistence(percentiles):
    """Calculate volatility regime persistence"""
    if len(percentiles) < 10:
        return np.nan
    
    regimes = np.where(percentiles > 75, 2, np.where(percentiles < 25, 0, 1))
    changes = np.sum(np.diff(regimes) != 0)
    persistence = 1 - (changes / len(regimes))
    return persistence

def comprehensive_stock_characterization(enriched_data):
    """Analyze inherent market characteristics of stocks"""
    
    characteristics = []
    
    for symbol, df in enriched_data.items():
        print(f"\nAnalyzing {symbol}...")
        
        for year in [2023, 2024, 2025]:
            year_df = df[df['year'] == year].dropna(subset=['close', 'returns'])
            
            if len(year_df) < 100:
                continue
            
            # Volatility characteristics
            atr = year_df['atr_14'].dropna()
            
            vol_characteristics = {
                'symbol': symbol,
                'year': year,
                'atr_mean': atr.mean(),
                'atr_std': atr.std(),
                'atr_cv': atr.std() / atr.mean() if atr.mean() > 0 else np.nan,
                'atr_skewness': skew(atr) if len(atr) > 3 else np.nan,
                'atr_kurtosis': kurtosis(atr) if len(atr) > 3 else np.nan,
                'vol_autocorr_lag1': year_df['tr'].autocorr(lag=1),
                'vol_autocorr_lag5': year_df['tr'].autocorr(lag=5),
            }
            
            # Price action characteristics
            returns = year_df['returns'].dropna()
            jb_stat, jb_pval = jarque_bera(returns) if len(returns) > 20 else (np.nan, np.nan)
            norm_stat, norm_pval = normaltest(returns) if len(returns) > 20 else (np.nan, np.nan)
            
            price_characteristics = {
                'return_mean': returns.mean() * 252,
                'return_std': returns.std() * np.sqrt(252),
                'return_skewness': skew(returns) if len(returns) > 3 else np.nan,
                'return_kurtosis': kurtosis(returns) if len(returns) > 3 else np.nan,
                'return_jarque_bera_stat': jb_stat,
                'return_jarque_bera_pval': jb_pval,
                'has_fat_tails': 'Yes' if jb_pval < 0.05 else 'No' if not np.isnan(jb_pval) else 'Unknown',
                'return_normality_pval': norm_pval,
                'is_normal': 'Yes' if norm_pval > 0.05 else 'No' if not np.isnan(norm_pval) else 'Unknown',
            }
            
            # Mean reversion characteristics
            autocorr_lags = {}
            for lag in [1, 2, 3, 5, 10, 20]:
                autocorr_lags[f'autocorr_lag{lag}'] = returns.autocorr(lag=lag)
            
            hurst = calculate_hurst_exponent(year_df['close'].values)
            half_life = calculate_half_life(returns.values)
            
            mean_reversion_characteristics = {
                **autocorr_lags,
                'hurst_exponent': hurst,
                'half_life_periods': half_life,
                'mean_reversion_strength': -np.mean([autocorr_lags[f'autocorr_lag{i}'] for i in [1, 2, 3, 5] if not np.isnan(autocorr_lags[f'autocorr_lag{i}'])]),
                'is_mean_reverting': 'Strong' if hurst < 0.4 else 'Weak' if hurst < 0.5 else 'Trending' if not np.isnan(hurst) else 'Unknown',
            }
            
            # S/R characteristics
            sr_range = year_df['sr_range_pct'].dropna()
            
            sr_characteristics = {
                'sr_range_mean': sr_range.mean(),
                'sr_range_std': sr_range.std(),
                'sr_range_cv': sr_range.std() / sr_range.mean() if sr_range.mean() > 0 else np.nan,
                'sr_stability_score': sr_range.mean() / sr_range.std() if sr_range.std() > 0 else np.nan,
                'price_near_support_pct': (year_df['distance_from_support'] < 1).sum() / len(year_df) * 100,
                'price_near_resistance_pct': (year_df['distance_from_resistance'] < 1).sum() / len(year_df) * 100,
                'sr_range_persistence': year_df['sr_range_pct'].autocorr(lag=1),
            }
            
            # Trend/range characteristics
            price_series = year_df['close'].values
            time_index = np.arange(len(price_series))
            trend_strength = abs(np.corrcoef(time_index, price_series)[0, 1]) if len(price_series) > 2 else np.nan
            
            bb_position = year_df['bollinger_position'].dropna()
            time_trending_pct = ((bb_position < 0.2) | (bb_position > 0.8)).sum() / len(bb_position) * 100 if len(bb_position) > 0 else np.nan
            time_ranging_pct = ((bb_position >= 0.4) & (bb_position <= 0.6)).sum() / len(bb_position) * 100 if len(bb_position) > 0 else np.nan
            
            trend_characteristics = {
                'trend_strength': trend_strength,
                'time_trending_pct': time_trending_pct,
                'time_ranging_pct': time_ranging_pct,
                'price_efficiency': calculate_price_efficiency(year_df['close'].values),
            }
            
            # Microstructure characteristics
            spread_proxy = year_df['range_pct']
            price_changes = year_df['close'].diff().dropna()
            
            microstructure_characteristics = {
                'avg_spread_pct': spread_proxy.mean(),
                'spread_volatility': spread_proxy.std(),
                'avg_tick_size': np.median(np.abs(price_changes[price_changes != 0])) if len(price_changes[price_changes != 0]) > 0 else np.nan,
                'zero_return_pct': (returns == 0).sum() / len(returns) * 100,
                'volume_mean': year_df['volume'].mean(),
                'volume_std': year_df['volume'].std(),
                'volume_cv': year_df['volume'].std() / year_df['volume'].mean() if year_df['volume'].mean() > 0 else np.nan,
                'volume_price_corr': year_df['volume'].corr(year_df['close']),
                'volume_volatility_corr': year_df['volume'].corr(year_df['atr_14']),
            }
            
            # Regime characteristics
            vol_percentile = year_df['volatility_percentile'].dropna()
            
            regime_characteristics = {
                'high_vol_regime_pct': (vol_percentile > 75).sum() / len(vol_percentile) * 100 if len(vol_percentile) > 0 else np.nan,
                'low_vol_regime_pct': (vol_percentile < 25).sum() / len(vol_percentile) * 100 if len(vol_percentile) > 0 else np.nan,
                'medium_vol_regime_pct': ((vol_percentile >= 25) & (vol_percentile <= 75)).sum() / len(vol_percentile) * 100 if len(vol_percentile) > 0 else np.nan,
                'regime_persistence': calculate_regime_persistence(vol_percentile.values) if len(vol_percentile) > 10 else np.nan,
            }
            
            # Combine all characteristics
            stock_char = {
                **vol_characteristics,
                **price_characteristics,
                **mean_reversion_characteristics,
                **sr_characteristics,
                **trend_characteristics,
                **microstructure_characteristics,
                **regime_characteristics
            }
            
            characteristics.append(stock_char)
    
    return pd.DataFrame(characteristics)

# ============================================================================
# RUN ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("COMPREHENSIVE STOCK CHARACTERIZATION - OHLCV ONLY")
print("="*80)

stock_characteristics = comprehensive_stock_characterization(enriched_data)

# Save results
stock_characteristics.to_csv('stock_characteristics_detailed.csv', index=False)
print("\n✓ Detailed characteristics saved to 'stock_characteristics_detailed.csv'")
print(f"✓ Analyzed {len(stock_characteristics)} stock-year combinations")
print(f"✓ Stocks analyzed: {stock_characteristics['symbol'].unique().tolist()}")

# ============================================================================
# STATISTICAL TESTS
# ============================================================================

print("\n" + "="*80)
print("STATISTICAL TESTS - COMPARING STOCK NATURES")
print("="*80)

# Test 1: Mean Reversion
print("\n" + "-"*80)
print("TEST 1: Mean Reversion Strength by Stock")
print("-"*80)

mr_summary = stock_characteristics.groupby('symbol').agg({
    'hurst_exponent': 'mean',
    'mean_reversion_strength': 'mean',
    'autocorr_lag1': 'mean',
    'half_life_periods': 'mean'
}).round(4)

mr_summary = mr_summary.sort_values('hurst_exponent')
print("\nHurst Exponent (Lower = More Mean Reverting):")
print(mr_summary)

print("\n" + "-"*40)
print("Stock Classification:")
print("-"*40)
for symbol in mr_summary.index:
    hurst = mr_summary.loc[symbol, 'hurst_exponent']
    if pd.notna(hurst):
        if hurst < 0.45:
            classification = "STRONG Mean Reversion ✓✓✓"
        elif hurst < 0.50:
            classification = "Moderate Mean Reversion ✓✓"
        elif hurst < 0.55:
            classification = "Weak/Random ✓"
        else:
            classification = "TRENDING ✗"
        print(f"{symbol:15s}: H={hurst:.3f} → {classification}")

# Test 2: Volatility
print("\n" + "-"*80)
print("TEST 2: Volatility Characteristics")
print("-"*80)

vol_summary = stock_characteristics.groupby('symbol').agg({
    'atr_mean': 'mean',
    'atr_cv': 'mean',
    'vol_autocorr_lag1': 'mean',
}).round(4)

vol_summary = vol_summary.sort_values('atr_cv')
print("\nVolatility Stability (Lower CV = More Stable):")
print(vol_summary)

# Test 3: S/R Quality
print("\n" + "-"*80)
print("TEST 3: Support/Resistance Quality")
print("-"*80)

sr_summary = stock_characteristics.groupby('symbol').agg({
    'sr_stability_score': 'mean',
    'sr_range_persistence': 'mean',
}).round(4)

sr_summary = sr_summary.sort_values('sr_stability_score', ascending=False)
print("\nS/R Stability (Higher = Better):")
print(sr_summary)

# Test 4: Ranging Behavior
print("\n" + "-"*80)
print("TEST 4: Range vs Trend Behavior")
print("-"*80)

trend_summary = stock_characteristics.groupby('symbol').agg({
    'time_ranging_pct': 'mean',
    'trend_strength': 'mean',
}).round(4)

trend_summary = trend_summary.sort_values('time_ranging_pct', ascending=False)
print("\nTime in Range (Higher = Better for Mean Reversion):")
print(trend_summary)

print("\n" + "="*80)
print("ANALYSIS COMPLETE!")
print("="*80)

LOADING STOCK DATA
✓ Loaded ADANIPORTS: 53855 candles
✓ Loaded AXISBANK: 53855 candles
✓ Loaded INFY: 53857 candles

CALCULATING FEATURES
✓ Calculated features for ADANIPORTS
✓ Calculated features for AXISBANK
✓ Calculated features for INFY

COMPREHENSIVE STOCK CHARACTERIZATION - OHLCV ONLY

Analyzing ADANIPORTS...

Analyzing AXISBANK...

Analyzing INFY...

✓ Detailed characteristics saved to 'stock_characteristics_detailed.csv'
✓ Analyzed 9 stock-year combinations
✓ Stocks analyzed: ['ADANIPORTS', 'AXISBANK', 'INFY']

STATISTICAL TESTS - COMPARING STOCK NATURES

--------------------------------------------------------------------------------
TEST 1: Mean Reversion Strength by Stock
--------------------------------------------------------------------------------

Hurst Exponent (Lower = More Mean Reverting):
            hurst_exponent  mean_reversion_strength  autocorr_lag1  \
symbol                                                               
AXISBANK            0.4958              

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import kstest, normaltest, jarque_bera, kurtosis, skew
import matplotlib.pyplot as plt
import seaborn as sns

# ============================================================================
# PURE OHLCV-BASED STATISTICAL ANALYSIS
# Stock Nature Characterization (No Backtest Data Needed)
# ============================================================================

def comprehensive_stock_characterization(enriched_data):
    """
    Analyze inherent market characteristics of stocks
    """
    
    characteristics = []
    
    for symbol, df in enriched_data.items():
        print(f"\nAnalyzing {symbol}...")
        
        for year in [2023, 2024, 2025]:
            year_df = df[df['year'] == year].dropna(subset=['close', 'returns'])
            
            if len(year_df) < 100:
                continue
            
            # ================================================================
            # 1. VOLATILITY CHARACTERISTICS
            # ================================================================
            atr = year_df['atr_14'].dropna()
            
            vol_characteristics = {
                'symbol': symbol,
                'year': year,
                
                # Basic volatility
                'atr_mean': atr.mean(),
                'atr_std': atr.std(),
                'atr_cv': atr.std() / atr.mean() if atr.mean() > 0 else np.nan,  # Coefficient of variation
                'atr_skewness': skew(atr) if len(atr) > 3 else np.nan,
                'atr_kurtosis': kurtosis(atr) if len(atr) > 3 else np.nan,
                
                # Volatility clustering (GARCH-like)
                'vol_autocorr_lag1': year_df['tr'].autocorr(lag=1),
                'vol_autocorr_lag5': year_df['tr'].autocorr(lag=5),
            }
            
            # ================================================================
            # 2. PRICE ACTION CHARACTERISTICS
            # ================================================================
            returns = year_df['returns'].dropna()
            
            # Only calculate if we have enough data
            jb_stat, jb_pval = jarque_bera(returns) if len(returns) > 20 else (np.nan, np.nan)
            norm_stat, norm_pval = normaltest(returns) if len(returns) > 20 else (np.nan, np.nan)
            
            price_characteristics = {
                # Return distribution
                'return_mean': returns.mean() * 252,  # Annualized
                'return_std': returns.std() * np.sqrt(252),  # Annualized
                'return_skewness': skew(returns) if len(returns) > 3 else np.nan,
                'return_kurtosis': kurtosis(returns) if len(returns) > 3 else np.nan,  # Excess kurtosis
                
                # Fat tails test
                'return_jarque_bera_stat': jb_stat,
                'return_jarque_bera_pval': jb_pval,
                'has_fat_tails': 'Yes' if jb_pval < 0.05 else 'No' if not np.isnan(jb_pval) else 'Unknown',
                
                # Normality test
                'return_normality_pval': norm_pval,
                'is_normal': 'Yes' if norm_pval > 0.05 else 'No' if not np.isnan(norm_pval) else 'Unknown',
            }
            
            # ================================================================
            # 3. MEAN REVERSION CHARACTERISTICS
            # ================================================================
            
            # Autocorrelation at multiple lags
            autocorr_lags = {}
            for lag in [1, 2, 3, 5, 10, 20]:
                autocorr = returns.autocorr(lag=lag)
                autocorr_lags[f'autocorr_lag{lag}'] = autocorr
            
            # Hurst exponent (mean reversion indicator)
            # H < 0.5 = mean reverting, H = 0.5 = random walk, H > 0.5 = trending
            hurst = calculate_hurst_exponent(year_df['close'].values)
            
            # Half-life of mean reversion
            half_life = calculate_half_life(returns.values)
            
            mean_reversion_characteristics = {
                **autocorr_lags,
                'hurst_exponent': hurst,
                'half_life_periods': half_life,
                'mean_reversion_strength': -np.mean([autocorr_lags[f'autocorr_lag{i}'] for i in [1, 2, 3, 5] if not np.isnan(autocorr_lags[f'autocorr_lag{i}'])]),
                'is_mean_reverting': 'Strong' if hurst < 0.4 else 'Weak' if hurst < 0.5 else 'Trending' if not np.isnan(hurst) else 'Unknown',
            }
            
            # ================================================================
            # 4. SUPPORT/RESISTANCE CHARACTERISTICS
            # ================================================================
            
            sr_range = year_df['sr_range_pct'].dropna()
            
            sr_characteristics = {
                # S/R level stability
                'sr_range_mean': sr_range.mean(),
                'sr_range_std': sr_range.std(),
                'sr_range_cv': sr_range.std() / sr_range.mean() if sr_range.mean() > 0 else np.nan,
                'sr_stability_score': sr_range.mean() / sr_range.std() if sr_range.std() > 0 else np.nan,
                
                # How often price is near S/R
                'price_near_support_pct': (year_df['distance_from_support'] < 1).sum() / len(year_df) * 100 if 'distance_from_support' in year_df else np.nan,
                'price_near_resistance_pct': (year_df['distance_from_resistance'] < 1).sum() / len(year_df) * 100 if 'distance_from_resistance' in year_df else np.nan,
                
                # S/R respect (how often they hold)
                'sr_range_persistence': year_df['sr_range_pct'].autocorr(lag=1),
            }
            
            # ================================================================
            # 5. TREND/RANGE CHARACTERISTICS
            # ================================================================
            
            # Calculate trend strength directly from price
            price_series = year_df['close'].values
            time_index = np.arange(len(price_series))
            trend_strength = abs(np.corrcoef(time_index, price_series)[0, 1]) if len(price_series) > 2 else np.nan
            
            # Percentage of time in trending vs ranging
            # Using bollinger band position if available
            if 'bollinger_position' in year_df.columns:
                bb_position = year_df['bollinger_position'].dropna()
                time_trending_pct = ((bb_position < 0.2) | (bb_position > 0.8)).sum() / len(bb_position) * 100 if len(bb_position) > 0 else np.nan
                time_ranging_pct = ((bb_position >= 0.4) & (bb_position <= 0.6)).sum() / len(bb_position) * 100 if len(bb_position) > 0 else np.nan
            else:
                # Alternative: use distance from moving average
                ma_20 = year_df['close'].rolling(20).mean()
                distance_from_ma = abs(year_df['close'] - ma_20) / year_df['close'] * 100
                time_trending_pct = (distance_from_ma > 2).sum() / len(year_df) * 100
                time_ranging_pct = (distance_from_ma < 1).sum() / len(year_df) * 100
            
            trend_characteristics = {
                'trend_strength': trend_strength,
                'time_trending_pct': time_trending_pct,
                'time_ranging_pct': time_ranging_pct,
                
                # Price efficiency (straight line / actual path)
                'price_efficiency': calculate_price_efficiency(year_df['close'].values),
            }
            
            # ================================================================
            # 6. MICROSTRUCTURE CHARACTERISTICS
            # ================================================================
            
            # Spread proxy (high - low as % of close)
            spread_proxy = year_df['range_pct']
            
            # Tick movement analysis
            price_changes = year_df['close'].diff().dropna()
            
            microstructure_characteristics = {
                # Liquidity proxies
                'avg_spread_pct': spread_proxy.mean(),
                'spread_volatility': spread_proxy.std(),
                
                # Price discreteness
                'avg_tick_size': np.median(np.abs(price_changes[price_changes != 0])) if len(price_changes[price_changes != 0]) > 0 else np.nan,
                'zero_return_pct': (returns == 0).sum() / len(returns) * 100,
                
                # Volume characteristics
                'volume_mean': year_df['volume'].mean(),
                'volume_std': year_df['volume'].std(),
                'volume_cv': year_df['volume'].std() / year_df['volume'].mean() if year_df['volume'].mean() > 0 else np.nan,
                
                # Volume-price relationship
                'volume_price_corr': year_df['volume'].corr(year_df['close']),
                'volume_volatility_corr': year_df['volume'].corr(year_df['atr_14']),
            }
            
            # ================================================================
            # 7. REGIME CHARACTERISTICS
            # ================================================================
            
            if 'volatility_percentile' in year_df.columns:
                vol_percentile = year_df['volatility_percentile'].dropna()
                
                regime_characteristics = {
                    # Volatility regime distribution
                    'high_vol_regime_pct': (vol_percentile > 75).sum() / len(vol_percentile) * 100 if len(vol_percentile) > 0 else np.nan,
                    'low_vol_regime_pct': (vol_percentile < 25).sum() / len(vol_percentile) * 100 if len(vol_percentile) > 0 else np.nan,
                    'medium_vol_regime_pct': ((vol_percentile >= 25) & (vol_percentile <= 75)).sum() / len(vol_percentile) * 100 if len(vol_percentile) > 0 else np.nan,
                    
                    # Regime persistence
                    'regime_persistence': calculate_regime_persistence(vol_percentile.values) if len(vol_percentile) > 10 else np.nan,
                }
            else:
                regime_characteristics = {
                    'high_vol_regime_pct': np.nan,
                    'low_vol_regime_pct': np.nan,
                    'medium_vol_regime_pct': np.nan,
                    'regime_persistence': np.nan,
                }
            
            # ================================================================
            # COMBINE ALL CHARACTERISTICS
            # ================================================================
            
            stock_char = {
                **vol_characteristics,
                **price_characteristics,
                **mean_reversion_characteristics,
                **sr_characteristics,
                **trend_characteristics,
                **microstructure_characteristics,
                **regime_characteristics
            }
            
            characteristics.append(stock_char)
    
    return pd.DataFrame(characteristics)


# ============================================================================
# HELPER FUNCTIONS FOR ADVANCED METRICS
# ============================================================================

def calculate_hurst_exponent(prices, max_lag=20):
    """
    Calculate Hurst Exponent
    H < 0.5: Mean reverting
    H = 0.5: Random walk
    H > 0.5: Trending
    """
    if len(prices) < 100:
        return np.nan
    
    lags = range(2, min(max_lag, len(prices) // 2))
    tau = []
    
    for lag in lags:
        # Calculate standard deviation of differences
        pp = np.subtract(prices[lag:], prices[:-lag])
        tau.append(np.std(pp))
    
    # Linear fit
    try:
        if len(tau) > 0:
            poly = np.polyfit(np.log(lags), np.log(tau), 1)
            return poly[0]
        else:
            return np.nan
    except:
        return np.nan


def calculate_half_life(returns):
    """
    Calculate half-life of mean reversion (Ornstein-Uhlenbeck)
    """
    if len(returns) < 50:
        return np.nan
    
    # Lag the returns
    returns_lag = np.roll(returns, 1)
    returns_lag[0] = 0
    
    # Regression
    returns_diff = returns - returns_lag
    
    try:
        # Use returns_lag as predictor
        from scipy import stats as sp_stats
        slope, _, _, _, _ = sp_stats.linregress(returns_lag[1:], returns_diff[1:])
        
        if slope >= 0:
            return np.nan
        
        half_life = -np.log(2) / slope
        return half_life if half_life > 0 and half_life < 1000 else np.nan
    except:
        return np.nan


def calculate_price_efficiency(prices):
    """
    Calculate how efficient price movement is
    1.0 = straight line, <1.0 = meandering
    """
    if len(prices) < 10:
        return np.nan
    
    # Distance from start to end
    direct_distance = abs(prices[-1] - prices[0])
    
    # Actual path length
    actual_distance = np.sum(np.abs(np.diff(prices)))
    
    if actual_distance == 0:
        return np.nan
    
    efficiency = direct_distance / actual_distance
    return efficiency


def calculate_regime_persistence(percentiles):
    """
    How long does volatility stay in same regime
    """
    if len(percentiles) < 10:
        return np.nan
    
    # Define regimes
    regimes = np.where(percentiles > 75, 2, np.where(percentiles < 25, 0, 1))
    
    # Count regime changes
    changes = np.sum(np.diff(regimes) != 0)
    
    # Persistence = inverse of change frequency
    persistence = 1 - (changes / len(regimes))
    return persistence


# ============================================================================
# RUN COMPREHENSIVE ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("COMPREHENSIVE STOCK CHARACTERIZATION - OHLCV ONLY")
print("="*80)

stock_characteristics = comprehensive_stock_characterization(enriched_data)

# Save to CSV for inspection
stock_characteristics.to_csv('stock_characteristics_detailed.csv', index=False)
print("\n✓ Detailed characteristics saved to 'stock_characteristics_detailed.csv'")

print(f"\n✓ Analyzed {len(stock_characteristics)} stock-year combinations")
print(f"✓ Stocks analyzed: {stock_characteristics['symbol'].unique().tolist()}")

# ============================================================================
# STATISTICAL COMPARISONS
# ============================================================================

print("\n" + "="*80)
print("STATISTICAL TESTS - COMPARING STOCK NATURES")
print("="*80)

# ----------------------------------------------------------------------------
# TEST 1: Mean Reversion Strength Comparison
# ----------------------------------------------------------------------------

print("\n" + "-"*80)
print("TEST 1: Mean Reversion Strength by Stock")
print("-"*80)

mr_summary = stock_characteristics.groupby('symbol').agg({
    'hurst_exponent': 'mean',
    'mean_reversion_strength': 'mean',
    'autocorr_lag1': 'mean',
    'half_life_periods': 'mean'
}).round(4)

mr_summary = mr_summary.sort_values('hurst_exponent')
print("\nHurst Exponent (Lower = More Mean Reverting):")
print(mr_summary)

print("\n" + "-"*40)
print("Stock Classification by Mean Reversion:")
print("-"*40)
for symbol in mr_summary.index:
    hurst = mr_summary.loc[symbol, 'hurst_exponent']
    if pd.notna(hurst):
        if hurst < 0.45:
            classification = "STRONG Mean Reversion ✓✓✓"
        elif hurst < 0.50:
            classification = "Moderate Mean Reversion ✓✓"
        elif hurst < 0.55:
            classification = "Weak Mean Reversion / Random ✓"
        else:
            classification = "TRENDING ✗"
        print(f"{symbol:15s}: H={hurst:.3f} → {classification}")
    else:
        print(f"{symbol:15s}: H=N/A → Insufficient data")

# Rest of the analysis continues...
# (I'll provide the rest in the next part to keep it manageable)


COMPREHENSIVE STOCK CHARACTERIZATION - OHLCV ONLY

Analyzing ADANIPORTS...

Analyzing AXISBANK...

Analyzing INFY...

✓ Detailed characteristics saved to 'stock_characteristics_detailed.csv'

✓ Analyzed 9 stock-year combinations
✓ Stocks analyzed: ['ADANIPORTS', 'AXISBANK', 'INFY']

STATISTICAL TESTS - COMPARING STOCK NATURES

--------------------------------------------------------------------------------
TEST 1: Mean Reversion Strength by Stock
--------------------------------------------------------------------------------

Hurst Exponent (Lower = More Mean Reverting):
            hurst_exponent  mean_reversion_strength  autocorr_lag1  \
symbol                                                               
AXISBANK            0.4958                   0.0082        -0.0221   
INFY                0.4998                  -0.0004         0.0035   
ADANIPORTS          0.5008                   0.0099        -0.0104   

            half_life_periods  
symbol                         
AXISB

In [6]:
df = pd.read_json("../results_json/results_json.json")

In [9]:
print(df[["ADANIPORTS_5", "AXISBANK_5", "INFY_5"]].to_json())

{"ADANIPORTS_5":{"metadata":{"filename":"ADANIPORTS","timeframe":5,"timestamp":"2026-01-27T16:01:16.669595","total_period_years":2.9103353867,"starting_capital":604.75,"ending_capital":1684.2231864286,"total_return_pct":178.4990800213,"cagr_pct":42.1813994967,"max_drawdown_pct":-1.7179517353},"yearly_performance":[{"year":2023,"Start Equity":604.75,"End Equity":899.6941964286,"Total Return (%)":48.771260261,"Max Drawdown (%)":-1.4082651339,"Trades":213.0,"Win Rate (%)":69.9530516432,"Avg Win":2.9625222819,"Avg Loss":-2.2886191183,"Sharpe Ratio":0.7328960446},{"year":2024,"Start Equity":899.6941964286,"End Equity":1256.9132185714,"Total Return (%)":39.7044933224,"Max Drawdown (%)":-1.7179517353,"Trades":220.0,"Win Rate (%)":66.8181818182,"Avg Win":4.5112333333,"Avg Loss":-4.1908531213,"Sharpe Ratio":0.5629066316},{"year":2025,"Start Equity":1256.9132185714,"End Equity":1684.2231864286,"Total Return (%)":33.9967757156,"Max Drawdown (%)":-1.2570854322,"Trades":213.0,"Win Rate (%)":69.0140